# Exploitation Zone - Model Predictiu

- exploitation zone del model predictiu
- preparation pipeline per taula d'entrenament del model --> cada zipcode és un indiv
    - Sales: 5 categories més comunes per zipcode, count vendes per zipcode, profit mitja per zipcode, mitjana num unitat per comanda per zipcode
    - Shops: 5shops més comunes per zipcode
    - Income: mitjana income per zipcode

In [ ]:
#!pip install pyspark
#!pip install delta-spark

import pyspark
from delta import *

#!wget -O "HR_comma_sep.csv" "https://mydisk.cs.upc.edu/s/3o33yciBHADiFCD/download/HR_comma_sep.csv"

builder = pyspark.sql.SparkSession.builder.appName("Shops_Deltalake") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
#Arxiu Parquet 
shops = spark.read.parquet("./datalake/shops_data/2024-04-17_shops_data.parquet")
income = spark.read.parquet("./datalake/income_data/2024-04-22_IRSIncomeByZipCode_NoStateTotalsNoSmallZips.parquet")
sales = spark.read.parquet("./datalake/sales_data/2024-04-22_SuperstoreSalesTraining.parquet")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, desc, row_number
from pyspark.sql.window import Window

# Calcular la media de ingresos por código postal
income_avg = income_data.groupBy("zipcode").agg(avg("income").alias("avg_income"))

# Determinar las tiendas más comunes por código postal
window = Window.partitionBy("zipcode").orderBy(desc("count"))
top_shops = sales_data.groupBy("zipcode", "shop").agg(count("*").alias("count")).\
    withColumn("rn", row_number().over(window)).filter(col("rn") <= 5)

# Calcular métricas de ventas por código postal y tienda
sales_metrics = sales_data.groupBy("zipcode", "shop", "category").\
    agg(count("*").alias("count"), avg("sales").alias("avg_sales"), avg("profit").alias("avg_profit"),
        avg("num_units").alias("avg_num_units"))

# Combinar todas las métricas en una sola tabla
result_table = sales_metrics.join(income_avg, "zipcode", "left").\
    join(top_shops, ["zipcode", "shop"], "left")

# Mostrar el resultado
result_table.show()

# Finalizar la sesión de Spark
spark.stop()
